You now know two kinds of regression and two kinds of classifier. So let's use that to compare models!

Comparing models is something data scientists do all the time. There's very rarely just one model that would be possible to run for a given situation, so learning to choose the best one is very important.

Here let's work on regression. Find a data set and build a KNN Regression and an OLS regression. Compare the two. How similar are they? Do they miss in different ways?

Create a Jupyter notebook with your models. At the end in a markdown cell write a few paragraphs to describe the models' behaviors and why you favor one model or the other. Try to determine whether there is a situation where you would change your mind, or whether one is unambiguously better than the other. Lastly, try to note what it is about the data that causes the better model to outperform the weaker model. 

In [40]:
import os
os.chdir('C:\\Users\\M246047\\Documents\\Python')
import numpy as np
import pandas as pd
import datetime as dt
import pylab
from matplotlib import pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')
import scipy as sc
from scipy.stats import ttest_ind
import re
import seaborn as sns
from sklearn import neighbors
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import normalize
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

spotifytop = pd.read_csv('spotifytop2018.csv')
music = pd.DataFrame(spotifytop)
print(music.columns)
print('\n')
music.describe()

Index(['id', 'name', 'artists', 'danceability', 'energy', 'key', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration_ms', 'time_signature'],
      dtype='object')




,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
count,100.00000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.00
mean,0.71646,0.659060,5.330000,-5.677640,0.590000,0.115569,0.195701,0.001584,0.158302,0.484443,119.904180,205206.780000,3.98
std,0.13107,0.145067,3.676447,1.777577,0.494311,0.104527,0.220946,0.013449,0.111662,0.206145,28.795984,40007.893404,0.20
min,0.25800,0.296000,0.000000,-10.109000,0.000000,0.023200,0.000282,0.000000,0.021500,0.079600,64.934000,95467.000000,3.00
25%,0.63550,0.562000,1.750000,-6.650500,0.000000,0.045350,0.040225,0.000000,0.094675,0.341000,95.730750,184680.000000,4.00
50%,0.73300,0.678000,5.000000,-5.566500,1.000000,0.074950,0.109000,0.000000,0.118500,0.470500,120.116000,205047.500000,4.00
75%,0.79825,0.772250,8.250000,-4.363750,1.000000,0.137000,0.247750,0.000031,0.170750,0.641500,140.022750,221493.250000,4.00
max,0.96400,0.909000,11.000000,-2.384000,1.000000,0.530000,0.934000,0.134000,0.636000,0.931000,198.075000,417920.000000,5.00


In [41]:
music['normalized_loudness'] = normalize(np.array(music["loudness"]).reshape(1,-1)).reshape(-1,1)
music['normalized_tempo'] = normalize(np.array(music["tempo"]).reshape(1,-1)).reshape(-1,1)

music.corr()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,normalized_loudness,normalized_tempo
danceability,1.000000,-0.072582,-0.051759,0.015517,-0.058019,0.227075,-0.134374,-0.066592,-0.038761,0.413855,-0.195012,-0.068368,0.119421,0.015517,-0.195012
energy,-0.072582,1.000000,-0.136345,0.732719,-0.117555,-0.073591,-0.421209,0.093684,0.050542,0.382434,0.062272,0.073017,0.255235,0.732719,0.062272
key,-0.051759,-0.136345,1.000000,-0.105309,-0.141568,0.019583,0.141590,-0.136607,-0.125443,-0.032622,0.003737,0.046144,-0.087096,-0.105309,0.003737
loudness,0.015517,0.732719,-0.105309,1.000000,-0.110178,-0.252037,-0.269742,0.036248,0.000006,0.407760,-0.035156,0.265310,0.072301,1.000000,-0.035156
mode,-0.058019,-0.117555,-0.141568,-0.110178,1.000000,-0.150076,-0.030028,0.089667,0.024428,-0.210599,-0.011911,0.055411,-0.083782,-0.110178,-0.011911
speechiness,0.227075,-0.073591,0.019583,-0.252037,-0.150076,1.000000,-0.081536,-0.069543,-0.099379,-0.051054,0.102999,-0.009856,0.235615,-0.252037,0.102999
acousticness,-0.134374,-0.421209,0.141590,-0.269742,-0.030028,-0.081536,1.000000,-0.089583,-0.150177,-0.020800,-0.158013,-0.069627,-0.158935,-0.269742,-0.158013
instrumentalness,-0.066592,0.093684,-0.136607,0.036248,0.089667,-0.069543,-0.089583,1.000000,-0.016249,-0.095123,0.178142,-0.045873,0.011894,0.036248,0.178142
liveness,-0.038761,0.050542,-0.125443,0.000006,0.024428,-0.099379,-0.150177,-0.016249,1.000000,-0.042612,-0.107652,-0.042942,-0.079558,0.000006,-0.107652
valence,0.413855,0.382434,-0.032622,0.407760,-0.210599,-0.051054,-0.020800,-0.095123,-0.042612,1.000000,-0.148423,-0.131901,0.223410,0.407760,-0.148423


## Predicting Energy with Loudness and Valence

Five neighbors

In [42]:
knn = neighbors.KNeighborsRegressor(n_neighbors=5)
Y = music.energy
X = music[['normalized_loudness','valence']]
knn.fit(X, Y)
Y_ = knn.predict(music[['normalized_loudness', 'valence']])
# Set up our prediction line.
T = np.arange(0, 50, 0.1)[:, np.newaxis]

# Trailing underscores are a common convention for a prediction.
knn_w = neighbors.KNeighborsRegressor(n_neighbors=5, weights='distance')
Y = music.energy
X = music[['normalized_loudness','valence']]
knn_w.fit(X, Y)
Y_ = knn_w.predict(music[['normalized_loudness', 'valence']])

score = cross_val_score(knn, X, Y, cv=5)
print("Unweighted Accuracy: %0.2f (+/- %0.2f)" % (score.mean(), score.std() * 2))
score_w = cross_val_score(knn_w, X, Y, cv=5)
print("Weighted Accuracy: %0.2f (+/- %0.2f)" % (score_w.mean(), score_w.std() * 2))

Unweighted Accuracy: 0.32 (+/- 0.48)
Weighted Accuracy: 0.33 (+/- 0.44)


Ten Neighbors

In [43]:
knn = neighbors.KNeighborsRegressor(n_neighbors=10)
Y = music.energy
X = music[['normalized_loudness','valence']]
knn.fit(X, Y)
Y_ = knn.predict(music[['normalized_loudness', 'valence']])
# Set up our prediction line.
T = np.arange(0, 50, 0.1)[:, np.newaxis]

# Trailing underscores are a common convention for a prediction.
knn_w = neighbors.KNeighborsRegressor(n_neighbors=10, weights='distance')
Y = music.energy
X = music[['normalized_loudness','valence']]
knn_w.fit(X, Y)
Y_ = knn_w.predict(music[['normalized_loudness', 'valence']])

score = cross_val_score(knn, X, Y, cv=5)
print("Unweighted Accuracy: %0.2f (+/- %0.2f)" % (score.mean(), score.std() * 2))
score_w = cross_val_score(knn_w, X, Y, cv=5)
print("Weighted Accuracy: %0.2f (+/- %0.2f)" % (score_w.mean(), score_w.std() * 2))

Unweighted Accuracy: 0.28 (+/- 0.46)
Weighted Accuracy: 0.30 (+/- 0.39)


OLS with training sets

In [46]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = .2, random_state = 465)
print("The number of observations in training set is {}".format(X_train.shape[0]))
print("The number of observations in test set is {}".format(X_test.shape[0]))

X_train_const = sm.add_constant(X_train)
X_test_const = sm.add_constant(X_test)

# We fit an OLS model using statsmodels
results_train = sm.OLS(Y_train, X_train_const).fit()


results_test = sm.OLS(Y_test, X_test_const).fit()
# We print the summary results
print(results_train.summary())
print(results_test.summary())

The number of observations in training set is 80
The number of observations in test set is 20
                            OLS Regression Results                            
Dep. Variable:                 energy   R-squared:                       0.598
Model:                            OLS   Adj. R-squared:                  0.587
Method:                 Least Squares   F-statistic:                     57.20
Date:                Thu, 07 Nov 2019   Prob (F-statistic):           5.96e-16
Time:                        21:21:13   Log-Likelihood:                 75.372
No. Observations:                  80   AIC:                            -144.7
Df Residuals:                      77   BIC:                            -137.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
----------------------------

These models are both pretty terrible, but I think I prefer the OLS to the KNN. While the OLS without training sets only accounts for 38.5% of the variance, the summary at least gives us information on how to improve the model. We can see that the p-vlaue for valence is well above the 0.05 threshold and as the adjusted R-squared value is lower than the R-squared, we can likely remove valence as a feature to obtain a better model. 

I also like that OLS gives an actual formula which gives me a more solid understanding of the data and how unobserved values might be predicted. 

I think I would prefer KNN if I knew my data had an unconventional pattern to it - if there were peaks and troughs that a correlation matrix wouldn't be able to grasp, but matching other datapoints did.

## Predicting Energy with Loudness

Five neighbors

In [28]:
knn = neighbors.KNeighborsRegressor(n_neighbors=5)
Y = music.energy
X = pd.DataFrame(music.normalized_loudness)
knn.fit(X, Y)
Y_ = knn.predict(X)
# Set up our prediction line.
T = np.arange(0, 50, 0.1)[:, np.newaxis]

# Trailing underscores are a common convention for a prediction.
knn_w = neighbors.KNeighborsRegressor(n_neighbors=5, weights='distance')
Y = music.energy
X = pd.DataFrame(music.normalized_loudness)
knn_w.fit(X, Y)
Y_ = knn_w.predict(X)

score = cross_val_score(knn, X, Y, cv=5)
print("Unweighted Accuracy: %0.2f (+/- %0.2f)" % (score.mean(), score.std() * 2))
score_w = cross_val_score(knn_w, X, Y, cv=5)
print("Weighted Accuracy: %0.2f (+/- %0.2f)" % (score_w.mean(), score_w.std() * 2))

Unweighted Accuracy: 0.26 (+/- 0.29)
Weighted Accuracy: 0.08 (+/- 0.29)


Ten neighbors

In [29]:
knn = neighbors.KNeighborsRegressor(n_neighbors=10)
Y = music.energy
X = pd.DataFrame(music.normalized_loudness)
knn.fit(X, Y)
Y_ = knn.predict(X)
# Set up our prediction line.
T = np.arange(0, 50, 0.1)[:, np.newaxis]

# Trailing underscores are a common convention for a prediction.
knn_w = neighbors.KNeighborsRegressor(n_neighbors=10, weights='distance')
Y = music.energy
X = pd.DataFrame(music.normalized_loudness)
knn_w.fit(X, Y)
Y_ = knn_w.predict(X)

score = cross_val_score(knn, X, Y, cv=5)
print("Unweighted Accuracy: %0.2f (+/- %0.2f)" % (score.mean(), score.std() * 2))
score_w = cross_val_score(knn_w, X, Y, cv=5)
print("Weighted Accuracy: %0.2f (+/- %0.2f)" % (score_w.mean(), score_w.std() * 2))

Unweighted Accuracy: 0.40 (+/- 0.38)
Weighted Accuracy: 0.18 (+/- 0.29)


## Predicting Energy with Acousticness

Five Neighbors

In [31]:
knn = neighbors.KNeighborsRegressor(n_neighbors=5)
Y = music.energy
X = pd.DataFrame(music.acousticness)
knn.fit(X, Y)
Y_ = knn.predict(X)
# Set up our prediction line.
T = np.arange(0, 50, 0.1)[:, np.newaxis]

# Trailing underscores are a common convention for a prediction.
knn_w = neighbors.KNeighborsRegressor(n_neighbors=5, weights='distance')
Y = music.energy
X = pd.DataFrame(music.acousticness)
knn_w.fit(X, Y)
Y_ = knn_w.predict(X)

score = cross_val_score(knn, X, Y, cv=5)
print("Unweighted Accuracy: %0.2f (+/- %0.2f)" % (score.mean(), score.std() * 2))
score_w = cross_val_score(knn_w, X, Y, cv=5)
print("Weighted Accuracy: %0.2f (+/- %0.2f)" % (score_w.mean(), score_w.std() * 2))

Unweighted Accuracy: -0.23 (+/- 0.69)
Weighted Accuracy: -0.64 (+/- 0.86)


In [32]:
knn = neighbors.KNeighborsRegressor(n_neighbors=10)
Y = music.energy
X = pd.DataFrame(music.acousticness)
knn.fit(X, Y)
Y_ = knn.predict(X)
# Set up our prediction line.
T = np.arange(0, 50, 0.1)[:, np.newaxis]

# Trailing underscores are a common convention for a prediction.
knn_w = neighbors.KNeighborsRegressor(n_neighbors=10, weights='distance')
Y = music.energy
X = pd.DataFrame(music.acousticness)
knn_w.fit(X, Y)
Y_ = knn_w.predict(X)

score = cross_val_score(knn, X, Y, cv=5)
print("Unweighted Accuracy: %0.2f (+/- %0.2f)" % (score.mean(), score.std() * 2))
score_w = cross_val_score(knn_w, X, Y, cv=5)
print("Weighted Accuracy: %0.2f (+/- %0.2f)" % (score_w.mean(), score_w.std() * 2))

Unweighted Accuracy: -0.03 (+/- 0.53)
Weighted Accuracy: -0.51 (+/- 0.74)
